![UPS](ups.png)
&nbsp;

**Nombre:** Jonathan Atancuri

**Asignatura:** Simulacion

# Examen Interciclo-Simulacion

### Objetivo:

* Consolidar los conocimientos adquiridos en clase para desarrollar simulaciones de eventos discretos.

### Introducción:

El golpe económico de la crisis sanitaria del corona virus no va a ser cosa de semanas, sino de meses. Dentro de una de las etapas importantes posteriores a las elecciones presidenciales son el análisis y tendencia que tiene el actual presidente. Para ello se plantea realizar un sistema de regresión que permita identificar cual es la tendencia en base al manejo de las redes sociales (Twitter y/o Facebook) del presidente [1].

Las regresiones lineales pueden aprenden por sí mismos y en este caso obtener automáticamente esa “recta o ecuación” que buscamos con la tendencia de predicción. Para ello se puede medir el error con respecto a los puntos de entrada y el valor “Y” de salida real [3].


#### Enunciado:

*	Diseñe y desarrolle un modelo y/o script que permita simular el siguiente caso real:
*	Obtener datos de tendencia de twitter o facebook(crawler o webscraping), para ello se puede obtener a través del API [4].
*	Title: Titulo del Post/Twitter
*	Word count: la cantidad de palabras del artículo,
*	number of Links: los enlaces externos que contiene,
*	number of comments: cantidad de comentarios,
*	number Shares: compartidos.
*	HashTag
*	Etc.

En base a ello, se pretende proponer y generar una predicción de cuántas veces será compartido un post/twitter utilizando regresión [2].


In [ ]:
import simpy as sp
import numpy as np
import tweepy as tp
import pandas as pd
import random as rd
import matplotlib.pyplot as pp

from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [ ]:
#Se definen las constantes necesarias para autenticarse en la API
CONSUMER_KEY = '7WjRyBLWCmtCutSig0mpqlec5'
CONSUMER_SECRET = 'BCdV2YThbvw6gC7tj5OZbjlR8DSU2gDFwaRjr7Xs1T5P4vmptM'
TOKEN_KEY = '1302969322533519360-tw5klydBCGiTfJ4VhsnJwuOEgK5OYI'
TOKEN_SECRET = 'E7PuklL3RpWVWOg91PfygcMC9absNBMX5xKm0YXHp2Kgk'

#Establecemos nuestras credenciales de desarrolladores
autenticacion = tp.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
autenticacion.set_access_token(TOKEN_KEY, TOKEN_SECRET)

#Accedemos a las funcionalidades de la API
twitter = tp.API(autenticacion, wait_on_rate_limit = True)
twitter

In [ ]:

#Definimos nuestros candidatos
USUARIOS = [
    #Presidente Republica
    '@LassoGuillermo', #Guillermo Lasso
]

USUARIOS

In [ ]:
#Definimos una lista donde se guardaran los datos
datos = []

#Realizamos la consulta y obtenemos los resultados
for usuario in USUARIOS:
    tweets = twitter.user_timeline(usuario, tweet_mode = 'extended')

    #Guardamos los datos en un diccionario
    for tweet in tweets:
        datos.append({
            'candidato': tweet.user.name,
            'fecha': tweet.created_at,
            'retweets': tweet.retweet_count,
            'favoritos': tweet.favorite_count,
            'tweet': tweet.full_text
    })

#Convertimos los datos en dataframe para facilitar el trabajo
datos = pd.DataFrame(data = datos, columns = ['candidato', 'fecha', 'retweets', 'favoritos','tweet'])

#Mostramos los 10 primero datos obtenidos
datos

In [ ]:
#Expresar los datos de fecha en número de dias desde inicio del año 2020
formato = '%Y-%m-%d'
fecha = datos['fecha']

datos['fecha'] = fecha.map(lambda x: 
    (datetime.strptime(str(x.date()), formato) - datetime.strptime('2021-01-01', formato)).days
)

datos

In [ ]:
#Graficamos las tendencia por grupos de datos a graficar
partidos = datos.groupby('candidato')

grafico, ejes = pp.subplots()

grafico.set_size_inches(20, 10)
for etiqueta, grupo in partidos:
    grupo.plot(x = 'fecha', y = 'retweets', ax = ejes,label = etiqueta)

pp.show()

In [ ]:
grafico = pp.figure(figsize = (15, 40))

i = 1
dias_totales = np.array(range(320, 400))

for etiqueta, grupo in partidos:
    #Obtención de las variables para el entrenamiento
    x = list(grupo.iloc[:, 1]) #Fecha (Número de día)
    y = list(grupo.iloc[:, 2]) #Retweets obtenidos

    #Creación y entranamiento del modelo
    
    modelo = LinearRegression()
    modelo.fit(np.array(x).reshape(-1, 1), y)

    #Establecemos el número de días que queremos predecir
    DIAS_PREDICCION = x[-1] + 100

    #Realizamos la prediccion
    prediccion = modelo.predict([[DIAS_PREDICCION]])
    prediccion = max(x) if int(max(x) + prediccion) <= 0 else int(max(x) + prediccion)
    
    candidato = grupo.iloc[0,0]
    
    parte = grafico.add_subplot(10, 2, i)
    parte.set_title(candidato + ' - Total de retweets: ' + str(prediccion))
    parte.scatter(x, y, color = 'green')
    parte.plot(dias_totales, modelo.predict(dias_totales.reshape(-1, 1)), color = 'red')
    parte.legend(['Retweets actuales', 'Retweets predecidos'])

    i += 1
    
pp.show()